In [ ]:
#this program uses the coordinate system and
#dikstra algorithm in order to find the 
#correct path between 2 points

from zumi.zumi import Zumi
import time
import math
from collections import defaultdict
from heapq import *


#file = open("speed_prediction.txt","r")
#data = file.readlines()
#speed_set = int(data[0])
#speed_cm_sec  = float(data[1])
#slope_int = float(data[2])

#these WILL vary for your Zumi!
speed_set = 40 #zumi speed set
speed_cm_sec  = 16.14 #cm per second
slope_int = -1.54 

zumi = Zumi()

#our position
current_x = 0 
current_y = 0

#length of the road in cm
l_road = 43
#road is eQUALLLL distance

'''

       Mini Map

    13- 14- 15- 16
    |   |   |   |
    9 - 10- 11- 12
    |   |   |   |
    5 - 6 - 7 - 8
    |   |   |   |
    1 - 2 - 3 - 4 

'''

#16 coordinates
location_coordinates =[
    ["1", 0*l_road,0*l_road],
    ["2", 1*l_road,0*l_road],
    ["3", 2*l_road,0*l_road], 
    ["4", 3*l_road,0*l_road],
    
    ["5", 0*l_road,1*l_road],
    ["6", 1*l_road,1*l_road],
    ["7", 2*l_road,1*l_road],
    ["8", 3*l_road,1*l_road],
    
    ["9",  0*l_road,2*l_road],
    ["10", 1*l_road,2*l_road],
    ["11", 2*l_road,2*l_road],
    ["12", 3*l_road,2*l_road],
    
    ["13", 0*l_road,3*l_road],
    ["14", 1*l_road,3*l_road],
    ["15", 2*l_road,3*l_road], 
    ["16", 3*l_road,3*l_road],
    ]        
    
'''
Mini Map
13- 14- 15- 16
|   |   |   |
9 - 10- 11- 12
|   |   |   |
5 - 6 - 7 - 8
|   |   |   |
1 - 2 - 3 - 4 

16 vertices
24 edges
'''
#24 edges
edges = [
    ("1", "2", l_road),
    ("1", "5", l_road),
    ("2", "3", l_road),
    ("2", "6", l_road),
    ("3", "4", l_road),
    ("3", "7", l_road),
    ("4", "8", l_road),

    ("5", "6" , l_road),
    ("5", "9" , l_road),
    ("6", "7" , l_road),
    ("6", "10", l_road),
    ("7", "11", l_road),
    ("7", "8" , l_road),
    ("8", "12", l_road),
    
    ("9", "10" , l_road),
    ("9", "13" , l_road),
    ("10", "14", l_road),
    ("10", "11", l_road),
    ("11", "15", l_road),
    ("11", "12", l_road),
    ("12", "16", l_road),
    
    ("13", "14", l_road),
    ("14", "15", l_road),
    ("15", "16", l_road),    
]


def dijkstra(edges, start, end):
   
    g = defaultdict(list)
    for l,r,c in edges:
        g[l].append((c,r))
    q, seen, mins = [(0,start,())], set(), {start: 0}
    while q:
        (cost,v1,path) = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            path += (v1, )
            
            if v1 == end: 
                return cost,path
            for c, v2 in g.get(v1, ()):
                if v2 in seen: continue
                prev = mins.get(v2, None)
                next = cost + c
                if prev is None or next < prev:
                    mins[v2] = next
                    heappush(q, (next, v2, path))
    print("error")
    return float("inf")


def drive_path(path_list, locations):
    for coordinate in path_list:
        ind = int(coordinate)
        move_to_coordinate(locations[ind-1][1],locations[ind-1][2])


#this function will move Zumi in the desired
#angle for a certain distance
#the accuracy of the distance traveled 
#is predicted by the given
#predicted speed and predicted y intercept
def move_cm(distance,angle):
    
    global speed_set,speed_cm_sec,slope_int
    
    speed = speed_set
    slope = speed_cm_sec
    y_intercept = slope_int
    
    # this is the speed Zumi 
    #travels at, in centimeters per second
    
    #how much time in seconds 
    #it takes to travel the distance in inches
    duration = (distance - y_intercept)/slope
    
    #make sure if there is no distance only turn
    if(distance<1):#distance is less than 1 cm
        zumi.turn(angle)
    #if there is a distance go at speed 40 at that angle
    else:
        #time.time returns the current time in seconds
        time_start = time.time()
        time_elapsed = 0
        while(duration > time_elapsed):
            #update the time that has passed
            time_elapsed = time.time()-time_start
            #take a step in that direction going forward
            zumi.drive_at_angle(127, 40, angle-1, 12, 0.1, 0, 0)

        #once done stop zumi
        zumi.hard_brake()

#this function will make Zumi move to a 
#desired coordinate in centimeters relative to the 
#origin or starting location of Zumi
#takes 2 parameters the desired x in cm and desired y in cm
#make sure that only Zumi moves it self otherwise you 
#will ruin its coordinate system
def move_to_coordinate(desired_x,desired_y):
    
    #we have to reference the global 
    #variables we want to change
    global current_x, current_y
    
    dx = desired_x - current_x
    dy = desired_y - current_y
    
    #find the angle
    angle = math.degrees(math.atan2(dy,dx)) 
    
    #find the distance to the coordinate
    distance = math.hypot(dx,dy)
    
    #update the coordinates
    #assume we reach the position
    current_x = desired_x
    current_y = desired_y

    zumi.turn(angle)
    
    time_start = time.time()
    move_cm(distance,angle)
    time_elapsed = time.time()-time_start
    return time_elapsed



In [ ]:
zumi = Zumi()

#recablibrate Zumi just in case
zumi.mpu.calibrate_MPU()
#reset the gyro list
zumi.reset_gyro()

#initiate our starting location to 0,0
current_x = 0
current_y = 0
#you must point Zumi at the X axis. this will be 0 degrees

try:
    '''
    Mini Map
    13- 14- 15- 16
    |   |   |   |
    9 - 10- 11- 12
    |   |   |   |
    5 - 6 - 7 - 8
    |   |   |   |
    1 - 2 - 3 - 4 

    16 vertices
    24 edges
    '''
    #move from position 1 to 8
    cost,path=dijkstra(edges, "1", "8")
    
    print(path," length ",cost)
    
    #actually make Zumi drive the path
    drive_path(path,location_coordinates)
finally:
    zumi.stop()